In [2]:
import cobra
import re
import json
import sys
import os
from cobra import Reaction
import pandas as pd
import numpy as np
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [4]:
m = cobra.io.read_sbml_model('Data/Model_specifications/iAM_Vc960_model.xml')

# gene dictionaries for mapping between ID and names
gene_name_dic = {}
for g in m.genes:
    gene_name_dic[g.name] = g.id
    
gene_name_dic2 = {}
for g in m.genes:
    gene_name_dic2[g.id] = g.name

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


# Load in data

In [ ]:
filename = 'coreclinic_bothclade_input_GSM_BD.xlsx'
BD12 = pd.read_excel(filename, 'BD-1.2')
BD2 = pd.read_excel(filename, 'BD-2')

lineage_snps = {'BD-1.2' : BD12, 
                'BD-2' : BD2}

In [ ]:
filename = 'intergenicSNPs_GSM_BD.xlsx'
BD12 = pd.read_excel(filename, 'BD-1.2')
BD2 = pd.read_excel(filename, 'BD-2')

lineage_intergenic = {'BD-1.2' : BD12, 
                'BD-2' : BD2}

In [7]:
filename = 'accessory_GSM_BD.xlsx'
BD12 = pd.read_excel(filename, 'BD-1.2')
BD2 = pd.read_excel(filename, 'BD-2')

lineage_accessory = {'BD-1.2' : BD12, 
                'BD-2' : BD2}

# Extract all of genes from SNPs

In [8]:
genes_snps = []
for a in lineage_snps:
    for ind, g in enumerate(lineage_snps[a]['GSM Model ID']):
        
        if str(g) != 'nan':
            for gene in m.genes:
                if gene.id in g:
                    genes_snps.append(gene.id)                    
                
genes_snps = list(set(genes_snps))

    
print('number of genes from SNPs results: ', len(genes_snps))
print(genes_snps)

In [ ]:
genes_snps_bd12 = []
a = 'BD-1.2'
for ind, g in enumerate(lineage_snps[a]['GSM Model ID']):
    if str(g) != 'nan':
        for gene in m.genes:
            if gene.id in g:
                genes_snps_bd12.append(gene.id)
                
genes_snps_bd12 = list(set(genes_snps_bd12))
print('BD-1.2', len(genes_snps_bd12))

genes_snps_bd2 = []
a = 'BD-2'
for ind, g in enumerate(lineage_snps[a]['GSM Model ID']):
    if str(g) != 'nan':
        for gene in m.genes:
            if gene.id in g:
                genes_snps_bd2.append(gene.id)

                
genes_snps_bd2 = list(set(genes_snps_bd2))
print('BD-2', len(genes_snps_bd2))


genes_snps_all = {}
genes_snps_all['BD-1.2'] = genes_snps_bd12
genes_snps_all['BD-2'] = genes_snps_bd2

In [1]:
# Extract all of genes from intergenic SNPs

In [ ]:
genes_intergenic = []
for a in lineage_intergenic:
    for ind, g in enumerate(lineage_intergenic[a]['GSM Model ID']):
        if str(g) != 'nan':
                for gene in m.genes:
                    if gene.id in g:
                        genes_intergenic.append(gene.id)
                
genes_intergenic = list(set(genes_intergenic))

print('number of genes from intergenic results: ', len(genes_intergenic))
print(genes_intergenic)

In [ ]:
genes_intergenic_bd12 = []
a = 'BD-1.2'
for ind, g in enumerate(lineage_intergenic[a]['GSM Model ID']):
    if str(g) != 'nan':
        for gene in m.genes:
            if gene.id in g:
                genes_intergenic_bd12.append(gene.id)
                
genes_intergenic_bd12 = list(set(genes_intergenic_bd12))
print('BD-1.2', len(genes_intergenic_bd12))

genes_intergenic_bd2 = []
a = 'BD-2'
for ind, g in enumerate(lineage_intergenic[a]['GSM Model ID']):
    if str(g) != 'nan':
        for gene in m.genes:
            if gene.id in g:
                genes_intergenic_bd2.append(gene.id)

                
genes_intergenic_bd2 = list(set(genes_intergenic_bd2))
print('BD-2', len(genes_intergenic_bd2))


genes_intergenic_all = {}
genes_intergenic_all['BD-1.2'] = genes_intergenic_bd12
genes_intergenic_all['BD-2'] = genes_intergenic_bd2

# Extract all of genes from accessory genes

In [10]:
genes_accessory = []
for a in lineage_accessory:
    for ind, g in enumerate(lineage_accessory[a]['GSM Model ID']):
        if str(g) != 'nan':
                for gene in m.genes:
                    if gene.id in g:
                        genes_accessory.append(gene.id)
                
genes_accessory = list(set(genes_accessory))

print('number of genes from accessory results: ', len(genes_accessory))
print(genes_accessory)

number of genes from accessory results:  4
['VC1091', 'VC2699', 'VC2285', 'VC2738']


In [ ]:
genes_accessory_bd12 = []
a = 'BD-1.2'
for ind, g in enumerate(lineage_accessory[a]['GSM Model ID']):
    if str(g) != 'nan':
        for gene in m.genes:
            if gene.id in g:
                genes_accessory_bd12.append(gene.id)
                
genes_accessory_bd12 = list(set(genes_accessory_bd12))
print('BD-1.2', len(genes_accessory_bd12))

genes_accessory_bd2 = []
a = 'BD-2'
for ind, g in enumerate(lineage_accessory[a]['GSM Model ID']):
    if str(g) != 'nan':
        for gene in m.genes:
            if gene.id in g:
                genes_accessory_bd2.append(gene.id)

                
genes_accessory_bd2 = list(set(genes_accessory_bd2))
print('BD-2', len(genes_accessory_bd2))


genes_accessory_all = {}
genes_accessory_all['BD-1.2'] = genes_accessory_bd12
genes_accessory_all['BD-2'] = genes_accessory_bd2

# Merge the genes together

In [12]:
genes_ALL = set(genes_snps + genes_intergenic + genes_accessory)

# Dataframe of the results

In [13]:
df_AB_snps = pd.DataFrame(index = genes_snps)
count = 0
for a in lineage_snps:
    a_col = []
    for g in genes_snps:
        presence = 0.0
        for ind, gene in enumerate(lineage_snps[a]['GSM Model ID']):
            if str(gene) != 'nan':
                if g in gene:
                    presence = 1
        a_col.append(presence)
    #print(len(a_col))
    df_AB_snps.insert(count, column = a, value = a_col)
    count += 1
    
# binary version of dataframe
df_AB_snps_bin = df_AB_snps.copy()
df_AB_snps_bin[df_AB_snps_bin> 0] = 1

In [ ]:
df_AB_accessory = pd.DataFrame(index = genes_accessory)
count = 0
for a in lineage_accessory:
    a_col = []
    for g in genes_accessory:
        presence = 0.0
        for ind, gene in enumerate(lineage_accessory[a]['GSM Model ID']):
            if str(gene) != 'nan':
                if g in gene:
                    presence = 1
        a_col.append(presence)
    #print(len(a_col))
    df_AB_accessory.insert(count, column = a, value = a_col)
    count += 1
    
# binary version of dataframe
df_AB_accessory_bin = df_AB_accessory.copy()
df_AB_accessory_bin[df_AB_accessory_bin> 0] = 1

In [14]:
df_AB_accessory = pd.DataFrame(index = genes_accessory)
count = 0
for a in clinic_accessory:
    a_col = []
    for g in genes_accessory:
        presence = 0.0
        for ind, gene in enumerate(clinic_accessory[a]['GSM Model ID']):
            if str(gene) != 'nan':
                if g in gene:
                    presence = 1
        a_col.append(presence)
    #print(len(a_col))
    df_AB_accessory.insert(count, column = a, value = a_col)
    count += 1
    
# binary version of dataframe
df_AB_accessory_bin = df_AB_accessory.copy()
df_AB_accessory_bin[df_AB_accessory_bin> 0] = 1

In [15]:
df_AB_all = pd.DataFrame(index = genes_ALL)
count = 0

# column for clinic just in snps
for ind, col_name in enumerate(df_AB_snps.columns):
    col = []
    for g in genes_ALL:
        if col_name not in df_AB_accessory.columns:
            if g in df_AB_snps.index:
                col.append(df_AB_snps[col_name][g])
            else:
                col.append(0)
        else:
            if g in df_AB_snps.index and g in df_AB_accessory.index:
                col.append(np.max([df_AB_snps[col_name][g], df_AB_accessory[col_name][g]]))
            elif g in df_AB_snps.index and not g in df_AB_accessory.index: 
                col.append(df_AB_snps[col_name][g])
            elif g not in df_AB_snps.index and g in df_AB_accessory.index: 
                col.append(df_AB_accessory[col_name][g])
            
            
    
    df_AB_all.insert(count, column = col_name, value = col)
    count += 1
      
        
# column for clinic just in accessory
for ind, col_name in enumerate(df_AB_accessory.columns):
    col = []
    if col_name not in df_AB_snps.columns:
        for g in genes_ALL:
            if g in df_AB_accessory.index:
                #print(df_AB_accessory[col_name][g])
                col.append(df_AB_accessory[col_name][g])
                
            else:
                #print(0)
                col.append(0)
                
        print(len(col))
        df_AB_all.insert(count, column = col_name, value = col)
        count += 1        
    
# binary 
df_AB_all_bin = df_AB_all.copy()
df_AB_all_bin[df_AB_all_bin> 0] = 1

df_AB_all.to_pickle('GSMGenes.pkl')
df_AB_all.to_csv('GSMGenes.csv')

In [ ]:
##FVA analysis

In [ ]:
## FVA on glucose only
m.reactions.get_by_id('EX_glc-D(e)').bounds = (-10.0, 1000.0)
m.reactions.get_by_id('EX_o2(e)').bounds = (-18.5, 1000.0)
        
for reaction in m.reactions:
    if reaction.bounds != (0.0, 0.0):
        if reaction.reversibility:
            reaction.bounds = (-1, 1)
        else:
            reaction.bounds = (0, 1)

fva_res = {}

# compute for WT first
res = cobra.flux_analysis.flux_variability_analysis(m, fraction_of_optimum = 0.0, processes = 12)
fva_res['WT'] = res.to_dict()

     
for ind, gene in enumerate(genes_ALL):
    
    with m:  
      m.genes.get_by_id(gene).knock_out()
      res = cobra.flux_analysis.flux_variability_analysis(m, fraction_of_optimum = 0.0, processes = 12)
    fva_res[gene] = res.to_dict()
    #print(gene)

  
    with open('fva_res_m9.json', 'w') as fn:
        json.dump(fva_res, fn)
    
    


In [ ]:
reactions = []
for r in m.reactions:
    reactions.append(r.id)
FVA_res = pd.DataFrame(index = reactions)

count = 0
for strain, res in FVA_res_ALL.items():
    results = pd.DataFrame.from_dict(res)
    #print(results.head)
    flux_span = pd.DataFrame(results['maximum']-results['minimum'])
    if strain != 'WT':
        FVA_res.insert(count, column = strain, value = flux_span.values)
    else:
        FVA_res.insert(count, column = strain, value = flux_span.values)
        count += 1
        

In [ ]:
IF_genes179 = pd.DataFrame(index = reactions)

IF_genes179.insert(0, column = 'WT', value = FVA_res['WT'])
count = 1
for col in FVA_res.columns:
    if col in genes_ALL:
        IF_genes179.insert(count, column = col, value = FVA_res[col])
        count += 1

In [ ]:
IF_genes179.to_csv('FVA.csv')

In [ ]:
m = cobra.io.read_sbml_model('Data/Model_specifications/iAM_Vc960_model.xml')

# gene dictionaries for mapping between ID and names
gene_name_dic = {}
for g in m.genes:
    gene_name_dic[g.name] = g.id
    
gene_name_dic2 = {}
for g in m.genes:
    gene_name_dic2[g.id] = g.name

In [ ]:
## Essentiality Rich Media

In [ ]:
m.reactions.get_by_id('EX_o2(e)').bounds = (-18.5, 0.0)

# get the carbon sources from the model
listSources=[]
for r in m.reactions:
    if 'EX_' in r.id:
        listSources.append(r.id)
                
m.reactions.get_by_id('EX_glc-D(e)').lower_bound = 0.0
m.reactions.get_by_id('EX_o2(e)').lower_bound = -18.5
medium = m.medium

CgrowthCapabilities = pd.DataFrame(columns=listSources)
# run in rich media
col = []
for source in listSources:
    m.reactions.get_by_id(source).lower_bound=-1.0

# get values for WT
    
try:
    sol_bio = m.optimize()
    if sol_bio.status != 'infeasible':
        col.append(sol_bio['vch_biomass'])
        WT_growth = sol_bio['vch_biomass']
    else:
        WT_growth = 0.0
        col.append(0.0)

except:
    WT_growth = 0.0
    col.append(0.0)
    
CgrowthCapabilities.insert(0, column = 'WT', value = col) 


# run for all knockouts
count = 1
KO_growth_rich = {}
for gene in genes_ALL:
    print(gene)
    with m: 
        col = []
        m.genes.get_by_id(gene).knock_out() 
        
        try:
            sol_bio = m.optimize()
            if sol_bio.status != 'infeasible':
                KO_growth_rich[gene] = sol_bio['vch_biomass']
                col.append(sol_bio['vch_biomass'])
            else:
                KO_growth_rich[gene] = 0.0
                col.append(0.0)

        except:
            KO_growth_rich[gene.id] = 0.0
            col.append(0.0)
            
    # turn transporter off
    if source not in medium.keys():
        m.reactions.get_by_id(source).lower_bound = 0.0
        
print(KO_growth_rich)
    

In [ ]:
m = cobra.io.read_sbml_model('Data/Model_specifications/iAM_Vc960_model.xml')

# gene dictionaries for mapping between ID and names
gene_name_dic = {}
for g in m.genes:
    gene_name_dic[g.name] = g.id
    
gene_name_dic2 = {}
for g in m.genes:
    gene_name_dic2[g.id] = g.name

In [ ]:
## Essentiality Minimal Media

In [ ]:
#In silico M9 minimal media + glucose aerobic;

m.reactions.get_by_id('EX_glc-D(e)').lower_bound = -10.0

m.reactions.get_by_id('EX_o2(e)').bounds = (-18.5, 0.0)
medium = m.medium


# get values for WT
   
try:
    sol_bio = m.optimize()
    if sol_bio.status != 'infeasible':
        WT_growth = sol_bio['vch_biomass']
    else:
        WT_growth = 0.0

except:
    WT_growth = 0.0

    
# run for all knockouts

KO_growth = {}
for gene in genes_ALL:
    print(gene)
    with m: 
        col = []
        m.genes.get_by_id(gene).knock_out() 
        
        try:
            sol_bio = m.optimize()
            if sol_bio.status != 'infeasible':
                KO_growth[gene] = sol_bio['vch_biomass']
            else:
                KO_growth[gene] = 0.0
                
        except:
            KO_growth[gene] = 0.0
print(KO_growth)

In [ ]:
## Auxotrophy

In [ ]:
m = cobra.io.read_sbml_model('Data/Model_specifications/iAM_Vc960_model.xml')
# aerobic
m.reactions.get_by_id('EX_o2(e)').bounds = (-18.5, 0.0)

# gene dictionaries for mapping between ID and names
gene_name_dic = {}
for g in m.genes:
    gene_name_dic[g.name] = g.id
    
gene_name_dic2 = {}
for g in m.genes:
    gene_name_dic2[g.id] = g.name
    
listSources=[]
for r in m.reactions:
    if 'EX_' in r.id:
        listSources.append(r.id)

#m.reactions.get_by_id('EX_glc-D(e)').bounds = (-10.0, 1000.0)
m.reactions.get_by_id('EX_o2(e)').lower_bound = -18.5
medium = m.medium

CgrowthCapabilities_auxo = pd.DataFrame(index=listSources)


# run for all knockouts
count = 0
for gene in growth_limiting_minimal:
    print(gene)
    
    with m: 
        col = []
        #gene_id = gene_name_dic[gene]
        m.genes.get_by_id(gene).knock_out() 
        
        for source in listSources:
            m.reactions.get_by_id(source).bounds= (-10.0, 1000.0)


            try:
                sol_bio = m.optimize()
                if sol_bio.status != 'infeasible':
                    col.append(sol_bio['vch_biomass'])
                else:
                    col.append(0.0)

            except:
                col.append(0.0)
    
    

            # turn transporter off
            if source not in medium.keys():
                m.reactions.get_by_id(source).bounds = (0.0, 0.0)
        
        CgrowthCapabilities_auxo.insert(count, column = gene, value = col)
        
print(CgrowthCapabilities_auxo)


In [ ]:
## Alternative Carbon

In [ ]:
# aerobic - 
m.reactions.get_by_id('EX_o2(e)').bounds = (-18.5, 0.0)

# get the carbon sources from the model
listPotCarbonSources=[]
for r in m.reactions:
    if 'EX_' in r.id:
        for met in r.metabolites:
            if 'C' in met.formula:
                listPotCarbonSources.append(r.id)
                
m.reactions.get_by_id('EX_glc-D(e)').lower_bound = 0.0
#m.reactions.get_by_id('sink_glu1sa[c]').lower_bound = 0.0
m.reactions.get_by_id('EX_o2(e)').lower_bound = -18.5
medium = m.medium

CgrowthCapabilities = pd.DataFrame(columns=listPotCarbonSources)
#CgrowthCapabilities.to_csv('WT_table.csv')

# get values for WT
 
col = []
for source in listPotCarbonSources:
    m.reactions.get_by_id(source).lower_bound=-10.0
    
    try:
        sol_bio = m.optimize()
        if sol_bio.status != 'infeasible':
            col.append(sol_bio['vch_biomass']) 
        else:
            col.append(0.0)

    except:
        col.append(0.0)
        
    #CgrowthCapabilities.insert(0, column = 'WT', value = col)

# run for all knockouts
count = 1
for gene in genes_ALL:
    
    with m: 
        col = []
        m.genes.get_by_id(gene).knock_out() 
        
        for source in listPotCarbonSources:
            m.reactions.get_by_id(source).lower_bound=-10.0


            try:
                sol_bio = m.optimize()
                if sol_bio.status != 'infeasible':
                    col.append(sol_bio['vch_biomass'])
                else:
                    col.append(0.0)

            except:
                col.append(0.0)
    
    

        # turn transporter off
        if source not in medium.keys():
            m.reactions.get_by_id(source).lower_bound = 0.0
        
        CgrowthCapabilities.insert(count, column = gene_name_dic2[gene], value = col)

print(CgrowthCapabilities)

In [ ]:
## Metabolite Yields

In [ ]:
m = cobra.io.read_sbml_model('Data/Model_specifications/iAM_Vc960_model.xml')

# gene dictionaries for mapping between ID and names
gene_name_dic = {}
for g in m.genes:
    gene_name_dic[g.name] = g.id
    
gene_name_dic2 = {}
for g in m.genes:
    gene_name_dic2[g.id] = g.name

In [ ]:
# turn off biomass as objective function
m.reactions.vch_biomass.objective_coefficient = 0.0

metabolites = []
for met in m.metabolites:
    metabolites.append(met.id)
    
# calculate for wild type
wt_yields = {}
with m:
    for met in metabolites:
        #print(met)
        dm_id = str(met) + '_dummy_drain'
        DM_met = Reaction(dm_id)
        DM_met.name = met + ' dummy drain'
        DM_met.lower_bound = 0.0
        DM_met.upper_bound = 0.0
        DM_met.add_metabolites({m.metabolites.get_by_id(met) : -1.0})
        m.add_reactions([DM_met])

        m.reactions.get_by_id(dm_id).bounds = (0.0, 1000.0)
        m.objective = m.reactions.get_by_id(dm_id)           
        # set direction
        m.objective_direction = 'max'
        try:
            sol = m.optimize()
            #print(sol[dm_id])
            if sol.status == 'optimal':
                if sol[dm_id] > 0.00001:
                    wt_yields[dm_id] = sol[dm_id]
                else:
                    wt_yields[dm_id] = 0.0

            else:
                wt_yields[dm_id] = 0.0
        except:
            wt_yields[dm_id] = 0.0



# repeat for all the genetic determinants
MetYields_all = pd.DataFrame(index = metabolites)
MetYields_all.insert(0, column = 'wt', value = wt_yields.values())

MetYields_changes_all = pd.DataFrame(index = metabolites)
MetYields_changes_all.insert(0, column = 'wt', value = [1]*len(wt_yields.keys()))

count = 1
for gene in genes_ALL:

    ko_yields = []
    ko_changes = []
    with m:
        m.genes.get_by_id(gene).knock_out()
        for met in metabolites:
            
            # add drain reaction for metabolite
            dm_id = str(met) + '_dummy_drain'
            DM_met = Reaction(dm_id)
            DM_met.name = met + ' dummy drain'
            DM_met.lower_bound = 0.0
            DM_met.upper_bound = 0.0
            DM_met.add_metabolites({m.metabolites.get_by_id(met) : -1.0})
            m.add_reactions([DM_met])

            m.reactions.get_by_id(dm_id).bounds = (0.0, 1000.0)
            m.objective = m.reactions.get_by_id(dm_id)   # set the maximisation of metabolite production flux as objective        
            # set direction
            m.objective_direction = 'max'
            try:
                sol = m.optimize()
                if sol.status == 'optimal':
                    if sol[dm_id] > 0.00001:
                        ko_yields.append(sol[dm_id])
                        ko_changes.append(sol[dm_id]/wt_yields[dm_id])
                    else:
                        ko_yields.append(0.0)
                        ko_changes.append(0.0)
                else:
                    ko_yields.append(0.0)
                    ko_changes.append(0.0)
            except:
                ko_yields.append(0.0)
                ko_changes.append(0.0)
    MetYields_all.insert(count, column = gene, value = ko_yields) # add the yields of metabolites to dataframe
    MetYields_changes_all.insert(count, column = gene, value = ko_changes) # add change in metabolite yield compared to WT metabolite yield
    count += 1


MetYields_all.to_pickle('MetYields_ALL.pkl', protocol = 2)
MetYields_changes_all.to_pickle('MetYields_changes_ALL.pkl', protocol = 2)

In [ ]:
# get the metabolite yields for just the genes in our list
MetYields179 = pd.DataFrame(index = MetYields_all.index)
count = 0
MetYields179.insert(0, column = 'wt', value = list(MetYields['wt'].values))
for ind, i in enumerate(MetYields.columns):
    if i in genes_ALL:
        count += 1
        MetYields179.insert(count, column = i, value = list(MetYields[i].values))
print(MetYields179)